#Beam Element - This notebook computes the mass matrix and internal forces and outputs them for later use

In [1]:
import numpy as np
import scipy as sp
import sympy as sym
import pickle

from scipy import linalg
from sympy import mpmath
from sympy import cos, sin

from IPython.display import display
from __future__ import division
from sympy.interactive import printing
printing.init_printing(use_latex='mathjax')

#### Define Needed Functions

In [2]:
def Skew_sym(v):
    """
    This function returns the skew symetric matrix 
    of the vector 'v' to affect the cross product of 'v'x'u'
    """
    v_matrix = sym.Matrix([[  0 , -v[2],  v[1]],
                          [v[2],     0, -v[0]],
                          [-v[1],  v[0],    0]])
    return v_matrix

In [3]:
def Axial_sym(A):
    a_vec = 1/2*sym.Matrix([A[2,1] - A[1,2], A[0,2] - A[2,0], A[1,0] - A[0,1]])
    return a_vec

In [4]:
def Rotate_sym(theta):
    """
    This function returns the symbolic rotation matrix 
    for the simple 2D-rotation about the third axis
    """
    R = sym.Matrix([[cos(theta),-sin(theta), 0],
                    [sin(theta), cos(theta), 0],[0,0,1]])
    return R

#### Define symbolic quantites

In [5]:
# nodal positons, velocities, and accelerations
"""
"""

# symbolic system parameters 
E, I, A, rho, x, l, tau_a = sym.symbols('E I A rho x l, tau_a')

# Beam coordinates at cross-section 1 and 2
r1 = sym.Matrix(['r_1x','r_1y'])
r2 = sym.Matrix(['r_2x','r_2y'])

# Deformation coordinates
# u = sym.Matrix(['u_1x','u_1y','u_1z','u_2x','u_2y','u_2z'])
# udot = sym.Matrix(['udot_1x','udot_1y','udot_1z','udot_2x','udot_2y','udot_2z'])
# uddot = sym.Matrix(['uddot_1x','uddot_1y','uddot_1z','uddot_2x','uddot_2y','uddot_2z'])
u_sym = sym.Matrix(['u_1x','u_1y','u_2x','u_2y'])
udot_sym = sym.Matrix(['udot_1x','udot_1y','udot_2x','udot_2y'])
uddot_sym = sym.Matrix(['uddot_1x','uddot_1y','uddot_2x','uddot_2y'])

u = sym.Matrix([u_sym[0:2,0],[0],u_sym[2:5,0],[0]])
udot = sym.Matrix([udot_sym[0:2,0],[0],udot_sym[2:5,0],[0]])
uddot = sym.Matrix([uddot_sym[0:2,0],[0],uddot_sym[2:5,0],[0]])

# theta = sym.Matrix(['theta_1x','theta_1y','theta_1z','theta_2x','theta_2y','theta_2z'])
# omega = sym.Matrix(['omega_1x','omega_1y','omega_1z','omega_2x','omega_2y','omega_2z'])
# alpha = sym.Matrix(['alpha_1x','alpha_1y','alpha_1z','alpha_2x','alpha_2y','alpha_2z'])
theta = sym.Matrix(['theta_1','theta_2'])
thetadot = sym.Matrix(['thetadot_1','thetadot_2'])
thetaddot = sym.Matrix(['thetaddot_1','thetaddot_2'])

# locating the point in the cross section
s1 = sym.Matrix([0, r1])
s2 = sym.Matrix([0, r2])
s = sym.Matrix.vstack(s1,s2)

# # define state variables
# e = sym.Matrix([u[0:3,0], theta[0:3,0], u[3:6,0], theta[3:6,0]])
# edot = sym.Matrix([udot[0:3,0], omega[0:3,0], udot[3:6,0], omega[3:6,0]])
# eddot = sym.Matrix([uddot[0:3,0], alpha[0:3,0], uddot[3:6,0], alpha[3:6,0]])
q = sym.Matrix([u[0:2,0], [theta[0]], u[3:5,0], [theta[1]]])
qdot = sym.Matrix([udot[0:2,0], [thetadot[0]], udot[3:5,0], [thetadot[1]]])
qddot = sym.Matrix([uddot[0:2,0], [thetaddot[0]], uddot[3:5,0], [thetaddot[1]]])

# edot = sym.Matrix([udot[0:2,0], omega[0:3,0], udot[3:6,0], omega[3:6,0]])
# eddot = sym.Matrix([uddot[0:2,0], alpha[0:3,0], uddot[3:6,0], alpha[3:6,0]])
# e = u
# edot = udot
# eddot = uddot
display([q,qdot,qddot])
display(u)
# display([theta,thetadot,thetaddot])

⎡⎡u₁ₓ ⎤, ⎡u̇₁ₓ ⎤, ⎡ü₁ₓ ⎤⎤
⎢⎢    ⎥  ⎢     ⎥  ⎢     ⎥⎥
⎢⎢u_1y⎥  ⎢u̇_1y⎥  ⎢ü_1y⎥⎥
⎢⎢    ⎥  ⎢     ⎥  ⎢     ⎥⎥
⎢⎢ θ₁ ⎥  ⎢ θ̇₁ ⎥  ⎢ θ̈₁ ⎥⎥
⎢⎢    ⎥  ⎢     ⎥  ⎢     ⎥⎥
⎢⎢u₂ₓ ⎥  ⎢u̇₂ₓ ⎥  ⎢ü₂ₓ ⎥⎥
⎢⎢    ⎥  ⎢     ⎥  ⎢     ⎥⎥
⎢⎢u_2y⎥  ⎢u̇_2y⎥  ⎢ü_2y⎥⎥
⎢⎢    ⎥  ⎢     ⎥  ⎢     ⎥⎥
⎣⎣ θ₂ ⎦  ⎣ θ̇₂ ⎦  ⎣ θ̈₂ ⎦⎦

⎡u₁ₓ ⎤
⎢    ⎥
⎢u_1y⎥
⎢    ⎥
⎢ 0  ⎥
⎢    ⎥
⎢u₂ₓ ⎥
⎢    ⎥
⎢u_2y⎥
⎢    ⎥
⎣ 0  ⎦

### Needed Matrix Quantities

In [6]:
# angular velocity for 2D case is trivial
omega1_skew = Skew_sym([0,0,thetadot[0]])
omega2_skew = Skew_sym([0,0,thetadot[1]])
alpha1_skew = Skew_sym([0,0,thetaddot[0]])
alpha2_skew = Skew_sym([0,0,thetaddot[1]])
# omega1_skew = Skew_sym(omega[0:3])
# omega2_skew = Skew_sym(omega[3:6])
# alpha1_skew = Skew_sym(alpha[0:3])
# alpha2_skew = Skew_sym(alpha[3:6])

# Rotation for a 2D problem
R1 = Rotate_sym(theta[0])
R2 = Rotate_sym(theta[1])
# display(R1,R2)

# "spatial" rotation matrix
R = sym.Matrix.vstack(sym.Matrix.hstack(R1,sym.zeros(3)), \
                      sym.Matrix.hstack(sym.zeros(3),R2))

# "spatial" angular velocity matrix
Omega_skew = sym.Matrix.vstack(sym.Matrix.hstack(omega1_skew,sym.zeros(3)), \
                               sym.Matrix.hstack(sym.zeros(3),omega2_skew))

# "spatial" angular acceleration matrix
Alpha_skew = sym.Matrix.vstack(sym.Matrix.hstack(alpha1_skew,sym.zeros(3)), \
                               sym.Matrix.hstack(sym.zeros(3),alpha2_skew))
# display(Omega)

### Define Kinematics

In [7]:
# Define velocity of element endpoints (nodes)
v = sym.simplify(udot + R*Omega_skew*s)
print('v = ')
display(v)

# Define acceleration of element endpoints (nodes)
a = sym.simplify(uddot + R*Omega_skew*Omega_skew*s + R*Alpha_skew*s)
print('\na = ')
display(a)

v = 


⎡-r₁ₓ⋅θ̇₁⋅cos(θ₁) + u̇₁ₓ ⎤
⎢                        ⎥
⎢-r₁ₓ⋅θ̇₁⋅sin(θ₁) + u̇_1y⎥
⎢                        ⎥
⎢           0            ⎥
⎢                        ⎥
⎢-r₂ₓ⋅θ̇₂⋅cos(θ₂) + u̇₂ₓ ⎥
⎢                        ⎥
⎢-r₂ₓ⋅θ̇₂⋅sin(θ₂) + u̇_2y⎥
⎢                        ⎥
⎣           0            ⎦


a = 


⎡                          2                ⎤
⎢-r₁ₓ⋅θ̈₁⋅cos(θ₁) + r₁ₓ⋅θ̇₁ ⋅sin(θ₁) + ü₁ₓ ⎥
⎢                                           ⎥
⎢                          2                ⎥
⎢-r₁ₓ⋅θ̈₁⋅sin(θ₁) - r₁ₓ⋅θ̇₁ ⋅cos(θ₁) + ü_1y⎥
⎢                                           ⎥
⎢                     0                     ⎥
⎢                                           ⎥
⎢                          2                ⎥
⎢-r₂ₓ⋅θ̈₂⋅cos(θ₂) + r₂ₓ⋅θ̇₂ ⋅sin(θ₂) + ü₂ₓ ⎥
⎢                                           ⎥
⎢                          2                ⎥
⎢-r₂ₓ⋅θ̈₂⋅sin(θ₂) - r₂ₓ⋅θ̇₂ ⋅cos(θ₂) + ü_2y⎥
⎢                                           ⎥
⎣                     0                     ⎦

### Compute the Mass Matrix

In [8]:
# Define shape function for element with one node at each end
h = sym.symarray('h', 2)

h[0] = sym.Rational(1,2)*(1 - x)
h[1] = sym.Rational(1,2)*(1 + x)

# Compute shape function matrix
H = sym.expand(sym.Matrix([h[0]*sym.eye(3), h[1]*sym.eye(3)])).T
print('\nH = ')
display(H)

# Define velocity of any point 
Vp = H*v
print('\nV = ')
display(Vp)

# Define velocity of any point 
Ap = H*a
# print('\nA = ')
# display(Accel)

# Compute partial velocities of the nodes
Vr = sym.Matrix([[sym.diff(Vp,qdot) for Vp in Vp] for qdot in qdot]).T
# v_r = H
print('\nVr = ')
display(Vr)
# print(Vr.shape)

# Compute mass matrix
M = sym.factor(sym.Matrix([[sym.expand(sym.integrate(Vr[:,i].dot(Ap)*rho*A,(x,0,l))).coeff(qddot[j]) \
                            for i in range(len(qddot))] for j in range(len(qddot))]))
# print('\nM = \n')
# display(M)
pickle.dump( M, open( "mass-matrix.dump", "wb" ) )


H = 


⎡  x   1                    x   1              ⎤
⎢- ─ + ─     0        0     ─ + ─    0      0  ⎥
⎢  2   2                    2   2              ⎥
⎢                                              ⎥
⎢           x   1                  x   1       ⎥
⎢   0     - ─ + ─     0       0    ─ + ─    0  ⎥
⎢           2   2                  2   2       ⎥
⎢                                              ⎥
⎢                    x   1                x   1⎥
⎢   0        0     - ─ + ─    0      0    ─ + ─⎥
⎣                    2   2                2   2⎦


V = 


⎡ ⎛  x   1⎞                             ⎛x   1⎞                           ⎤
⎢ ⎜- ─ + ─⎟⋅(-r₁ₓ⋅θ̇₁⋅cos(θ₁) + u̇₁ₓ) + ⎜─ + ─⎟⋅(-r₂ₓ⋅θ̇₂⋅cos(θ₂) + u̇₂ₓ) ⎥
⎢ ⎝  2   2⎠                             ⎝2   2⎠                           ⎥
⎢                                                                         ⎥
⎢⎛  x   1⎞                              ⎛x   1⎞                           ⎥
⎢⎜- ─ + ─⎟⋅(-r₁ₓ⋅θ̇₁⋅sin(θ₁) + u̇_1y) + ⎜─ + ─⎟⋅(-r₂ₓ⋅θ̇₂⋅sin(θ₂) + u̇_2y)⎥
⎢⎝  2   2⎠                              ⎝2   2⎠                           ⎥
⎢                                                                         ⎥
⎣                                    0                                    ⎦


Vr = 


⎡  x   1                ⎛  x   1⎞          x   1              ⎛x   1⎞        ⎤
⎢- ─ + ─     0     -r₁ₓ⋅⎜- ─ + ─⎟⋅cos(θ₁)  ─ + ─    0    -r₂ₓ⋅⎜─ + ─⎟⋅cos(θ₂)⎥
⎢  2   2                ⎝  2   2⎠          2   2              ⎝2   2⎠        ⎥
⎢                                                                            ⎥
⎢           x   1       ⎛  x   1⎞                 x   1       ⎛x   1⎞        ⎥
⎢   0     - ─ + ─  -r₁ₓ⋅⎜- ─ + ─⎟⋅sin(θ₁)    0    ─ + ─  -r₂ₓ⋅⎜─ + ─⎟⋅sin(θ₂)⎥
⎢           2   2       ⎝  2   2⎠                 2   2       ⎝2   2⎠        ⎥
⎢                                                                            ⎥
⎣   0        0               0               0      0             0          ⎦

In [9]:
print('M_11 = ')
display(M[0:3,0:3])

print('\nM_22 = ')
display(M[3:6,3:6])

print('\nM_12 = ')
display(M[0:3,3:6])
print('\nM_21.T = ')
display(M[3:6,0:3].T)

M_11 = 


⎡             ⎛ 2          ⎞                                                  
⎢       A⋅l⋅ρ⋅⎝l  - 3⋅l + 3⎠                                                  
⎢       ────────────────────                         0                        
⎢                12                                                           
⎢                                                                             
⎢                                                 ⎛ 2          ⎞              
⎢                                           A⋅l⋅ρ⋅⎝l  - 3⋅l + 3⎠              
⎢                0                          ────────────────────              
⎢                                                    12                       
⎢                                                                             
⎢           ⎛ 2          ⎞                      ⎛ 2          ⎞                
⎢-A⋅l⋅r₁ₓ⋅ρ⋅⎝l  - 3⋅l + 3⎠⋅cos(θ₁)   -A⋅l⋅r₁ₓ⋅ρ⋅⎝l  - 3⋅l + 3⎠⋅sin(θ₁)   A⋅l⋅r
⎢──────────────────────────────────  ───────────────


M_22 = 


⎡             ⎛ 2          ⎞                                                  
⎢       A⋅l⋅ρ⋅⎝l  + 3⋅l + 3⎠                                                  
⎢       ────────────────────                         0                        
⎢                12                                                           
⎢                                                                             
⎢                                                 ⎛ 2          ⎞              
⎢                                           A⋅l⋅ρ⋅⎝l  + 3⋅l + 3⎠              
⎢                0                          ────────────────────              
⎢                                                    12                       
⎢                                                                             
⎢           ⎛ 2          ⎞                      ⎛ 2          ⎞                
⎢-A⋅l⋅r₂ₓ⋅ρ⋅⎝l  + 3⋅l + 3⎠⋅cos(θ₂)   -A⋅l⋅r₂ₓ⋅ρ⋅⎝l  + 3⋅l + 3⎠⋅sin(θ₂)   A⋅l⋅r
⎢──────────────────────────────────  ───────────────


M_12 = 


⎡            ⎛ 2    ⎞                                                         
⎢     -A⋅l⋅ρ⋅⎝l  - 3⎠                                                     A⋅l⋅
⎢     ────────────────                   0                                ────
⎢            12                                                               
⎢                                                                             
⎢                                        ⎛ 2    ⎞                             
⎢                                 -A⋅l⋅ρ⋅⎝l  - 3⎠                         A⋅l⋅
⎢            0                    ────────────────                        ────
⎢                                        12                                   
⎢                                                                             
⎢          ⎛ 2    ⎞                    ⎛ 2    ⎞                         ⎛ 2   
⎢A⋅l⋅r₁ₓ⋅ρ⋅⎝l  - 3⎠⋅cos(θ₁)  A⋅l⋅r₁ₓ⋅ρ⋅⎝l  - 3⎠⋅sin(θ₁)  -A⋅l⋅r₁ₓ⋅r₂ₓ⋅ρ⋅⎝l  - 
⎢──────────────────────────  ───────────────────────


M_21.T = 


⎡            ⎛ 2    ⎞                                                         
⎢     -A⋅l⋅ρ⋅⎝l  - 3⎠                                                     A⋅l⋅
⎢     ────────────────                   0                                ────
⎢            12                                                               
⎢                                                                             
⎢                                        ⎛ 2    ⎞                             
⎢                                 -A⋅l⋅ρ⋅⎝l  - 3⎠                         A⋅l⋅
⎢            0                    ────────────────                        ────
⎢                                        12                                   
⎢                                                                             
⎢          ⎛ 2    ⎞                    ⎛ 2    ⎞                         ⎛ 2   
⎢A⋅l⋅r₁ₓ⋅ρ⋅⎝l  - 3⎠⋅cos(θ₁)  A⋅l⋅r₁ₓ⋅ρ⋅⎝l  - 3⎠⋅sin(θ₁)  -A⋅l⋅r₁ₓ⋅r₂ₓ⋅ρ⋅⎝l  - 
⎢──────────────────────────  ───────────────────────

### Compute Internal forces 

#### Transverse (Bending) Strain

In [10]:
''' Interpolate Finite Rotation Between Nodes (Interpolation of rotation and motion - Olivier A. Bauchau · Shilei Han)
    - Algorithm 2 (Cayley’s decomposition) is used to Extract Orthogonal Rotation Matrix R
    - Algorithm 4 is used to interpolate curvature '''
T = sym.simplify(h[0]*R1 + h[1]*R2)
# display(T)
A = (T + sym.eye(3))*(T + sym.eye(3)).inv()
# display(A)
cpar = Axial_sym(A)
# display(cpar)
temp = cpar.T*cpar
cpar_skew = Skew_sym(cpar)
R = 1/(temp[0]+1)*((1+temp[0])*sym.eye(3) + 2*cpar_skew + 2*cpar_skew*cpar_skew)
dT = h[0].diff(x)*R1 + h[1].diff(x)*R2
# display(dT)
kappa = Axial_sym(dT*R.T)
# display(kappa)

#### Axial Strain

In [11]:
epsilon = 

SyntaxError: invalid syntax (<ipython-input-11-97695ad103bf>, line 1)

In [12]:
H.diff(x).T*H.diff(x)

⎡1/4    0     0    -1/4   0     0  ⎤
⎢                                  ⎥
⎢ 0    1/4    0     0    -1/4   0  ⎥
⎢                                  ⎥
⎢ 0     0    1/4    0     0    -1/4⎥
⎢                                  ⎥
⎢-1/4   0     0    1/4    0     0  ⎥
⎢                                  ⎥
⎢ 0    -1/4   0     0    1/4    0  ⎥
⎢                                  ⎥
⎣ 0     0    -1/4   0     0    1/4 ⎦

In [ ]:
# Test my Axial Function
# A = sym.MatrixSymbol('A',3,3)
# A = sym.Matrix(A)
# display(A)
# Skew_sym(Axial_sym(A)) - 1/2*(A - A.T)

#### Compute Longitudinal Strain Energy

#### Compute Transverse Strain Energy

#### Compute Internal Forces $Q_e = \frac{\partial U}{\partial e}$

### Applied and body force vector